In [5]:
import os
import mysql.connector
import csv
import requests
import os

from tqdm import tqdm
import cv2
import numpy as np
from shutil import copyfile
import subprocess, shlex


In [20]:

for root,dir,files in os.walk("/home/jy/projects/yolo/dataprepare/notvalid/"):
    if dir!=[]:
        dirs = dir



In [22]:
format_strings = ','.join(['%s'] * len(dirs))

In [23]:
% format_strings, tuple(list_of_ids)

'%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s'

In [38]:
sql


"select d.device_number,d.video_url from ( select a.device_name,a.device_number,b.* from camera.video_config as a left join ( select c.* from ( select * from video_0.video_data_motion_0 union all select * from video_0.video_data_motion_2 union all select * from video_1.video_data_motion_1 union all select * from video_1.video_data_motion_3 ) as c ) as b on a.device_number=b.device_serial where a.video_stream_url !=  and b.file_size>10000 and INSTR(a.name, 口 ) and b.start_date>'2020-09-26 00:00:00' and b.start_date <'2020-09-26 23:59:00') d where d.device_number IN  "

In [2]:
 

mydb = mysql.connector.connect(host="81.68.190.21", port='30203',user='root',password='iLabService123')

sql = "select d.device_number,d.video_url from ( select a.device_name,a.device_number,b.* " \
      "from camera.video_config as a left join ( select c.* from ( select * from video_0.video_data_motion_0 union all " \
      "select * from video_0.video_data_motion_2 union all " \
      "select * from video_1.video_data_motion_1 union all " \
      "select * from video_1.video_data_motion_3 ) as c ) as b " \
      "on a.device_number=b.device_serial " \
      "where a.video_stream_url !={arg1} and b.file_size>10000 and INSTR(a.name, {arg2} ) "\
    "and b.start_date>'2020-09-26 00:00:00' and b.start_date <'2020-09-26 23:59:00') d where d.device_number IN ({arg3})" 


arg1 = ""
arg2 = "口"

format_strings = ','.join(['%s'] * len(dirs))

arg3 = ("", "口",format_strings, tuple(dirs))

arg3 = str(arg3)  # Convert to string
 
sql = sql.format(arg1=arg1, arg2=arg2,arg3 = arg3)
 

print(sql)






NameError: name 'dirs' is not defined

In [15]:

with open("useway_1_needtofilter.txt","r",newline='') as file:
    reader = csv.reader(file,delimiter=',')
    for row in tqdm(reader):

        name = row[1].split("/")[-1]
        path = "/home/jy/projects/imgs/dataprepare/useway_1_needtofilter/"+row[0]+"/"+name
 
        r = requests.get(row[1], allow_redirects=True)
        if not os.path.exists("/home/jy/projects/imgs/dataprepare/useway_1_needtofilter/"+str(row[0])+"/"):
            os.makedirs("/home/jy/projects/imgs/dataprepare/useway_1_needtofilter/"+str(row[0])+"/")
        if not os.path.exists("/home/jy/projects/imgs/dataprepare/useway_1_needtofilter/"+str(row[0])+"/"+name):
            open(path, 'wb').write(r.content)    

0it [00:00, ?it/s]


ProxyError: HTTPSConnectionPool(host='prod-jiandu-shanghai.oss-cn-shanghai.aliyuncs.com', port=443): Max retries exceeded with url: /132530763_2020-11-10_00-11-33.mp4 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f69dc495390>: Failed to establish a new connection: [Errno 111] Connection refused',)))

In [10]:
 

with open("invalidpatch.csv","r",newline='') as file:
    reader = csv.reader(file,delimiter=',')
    for row in tqdm(reader):
 
        name = row[1].split("/")[-1]
        path = "/home/jy/projects/yolo/dataprepare/patch1/"+row[0]+"/"+name
        r = requests.get(row[1], allow_redirects=True)
        if not os.path.exists("/home/jy/projects/yolo/dataprepare/patch1/"+row[0]+"/"):
            os.makedirs("/home/jy/projects/yolo/dataprepare/patch1/"+row[0]+"/")
        if not os.path.exists("/home/jy/projects/yolo/dataprepare/patch1/"+row[0]+"/"+name):
            open(path, 'wb').write(r.content)


378it [12:00,  1.91s/it]


In [74]:
for root,dir,files in os.walk("/home/jy/projects/yolo/dataprepare/notvalid/"):
    for d in dir:
        for rootd,dird,filesd in os.walk(os.path.join(root,d)):
            if len(filesd)>0:
                vcap = cv2.VideoCapture(os.path.join(rootd,filesd[0]))
                fps = vcap.get(cv2.CAP_PROP_FPS)    

                frame_count = int(vcap.get(cv2.CAP_PROP_FRAME_COUNT))
                duration = frame_count/fps

                
                total_frames = vcap.get(7)
                print(total_frames)
                step = np.floor(total_frames/120)
                cout = 0
                img_array=[]
                width  = vcap.get(3) # float
                height = vcap.get(4)
                 
                size = (width,height)
 
                while True:
                    ret,frame = vcap.read()
                    if cout % step==0:
                        if ret==True:
                            img_array.append(frame)
 
                    if len(img_array)==120:
                        break
                    cout+=1
 
                print(filesd[0]+'.avi')
                out = filesd[0]+'.avi'
                fourcc = cv2.VideoWriter_fourcc(*'XVID')

                out = cv2.VideoWriter(filesd[0],fourcc=fourcc,fps=10, frameSize=size,isColor=True)
                for i in range(len(img_array)):
                    out.write(img_array[i])
                out.release() 
                
                
                
                
#             for f in filesd:
#                 print(os.path.join(root,f))
#         if file.split('.')[-1]=="mp4":
#              # 0=camera
#             width  = vcap.get(3) # float
#             height = vcap.get(4) 
#             total_frames = cap.get(7)         
#             img_array=[]
#             while True:
#                 ret,frame = vcap.read()
#                 if ret==True:
#                       img_array.append(img)
# out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
# for i in range(len(img_array)):
#     out.write(img_array[i])
# out.release() 

8160.0
D81142426_2020_09_25_06_45_32.mp4.avi


TypeError: Required argument 'apiPreference' (pos 2) not found

In [2]:

import cv2

In [10]:
for root,dir,files in os.walk("/home/jy/projects/yolo/dataprepare/valid/"):
    for root,dir,files in os.walk(root):
        if len(files)>1:
            for i in range(len(files)):
                if i == 0:
                    print(os.path.join(root,files[i]))
                    vcap = cv2.VideoCapture(os.path.join(root,files[i])) # 0=camera

                    width  = vcap.get(3) # float
                    height = vcap.get(4) 
                else:
                    

                    cap = cv2.VideoCapture(os.path.join(root,files[i]))

                    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
                    out = cv2.VideoWriter(os.path.join(root,'merge.mp4'),None,fourcc, 5, (int(width),int(height)),True)
                    while True:
                        ret, frame = cap.read()
                        if ret == True:
                            b = cv2.resize(frame,(int(width),int(height)),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
                            out.write(b)
                        else:
                            break

                    cap.release()
                    out.release()
                    cv2.destroyAllWindows()

/home/jy/projects/yolo/dataprepare/valid/D81142872/D81142872_2020_09_26_03_59_24.mp4
/home/jy/projects/yolo/dataprepare/valid/D81142586/D81142586_2020_09_26_07_18_16.mp4
/home/jy/projects/yolo/dataprepare/valid/D81141444/D81141444_2020_09_26_09_42_01.mp4


KeyboardInterrupt: 

In [27]:
for root,dir,files in os.walk("/home/jy/projects/yolo/dataprepare/valid/"):
    for root,dir,files in os.walk(root):
        if len(files)>1:
 #             for i,f in enumerate( files):
#                 files[i] = os.path.join(root,f)
#             f = " ".join(files)
# #             print(f)
# #             for file in files:
                
# #                 with open("list.txt","w") as ls:
# #                     path = os.path.join(root,file)
                    
# #                     ls.write(path)
# #                     ls.write("\n")
                
                
#             stringa = "ffmpeg -f concat -safe 0  -i "+f
#             stringb= " -c copy "
#             outname = file.split("_")[0]+".mp4"
#             stringc = os.path.join(root,outname)
#             stringf = stringa+stringb+stringc
# #             with open("list.txt","r") as ls:
# #                 print(ls.readline())
# #             os.system(stringf)
#             print(stringf)
# #             os.remove("list.txt")

downloadvideo.ipynb  invalidpatch.csv  list.txt  notvalid  types.ods  valid


TypeError: system() missing required argument 'command' (pos 1)

In [23]:
!ls

downloadvideo.ipynb  invalidpatch.csv  list.txt  notvalid  types.ods  valid


In [33]:
def concatenate():
	stringa = "ffmpeg -i \"concat:"
	elenco_video = glob.glob("*.mp4")
	elenco_file_temp = []
	for f in elenco_video:
		file = "temp" + str(elenco_video.index(f) + 1) + ".ts"
		os.system("ffmpeg -i " + f + " -c copy -bsf:v h264_mp4toannexb -f mpegts " + file)
		elenco_file_temp.append(file)
	print(elenco_file_temp)
	for f in elenco_file_temp:
		stringa += f
		if elenco_file_temp.index(f) != len(elenco_file_temp)-1:
			stringa += "|"
		else:
			stringa += "\" -c copy  -bsf:a aac_adtstoasc output.mp4"
	print(stringa)
	os.system(stringa)
 
concatenate()

[]
ffmpeg -i "concat:


In [4]:
import glob

In [42]:
np.random.seed(123)

In [43]:

def genData(n):
    mu = np.array([-2.0,3.0]) #shape 
    
 
    A = np.array([[1.0,2.0],[3.0,4.0]]) 
 
    X = np.random.normal(size = n*2)
 
    X = X.reshape((n,2))
 
    X = X.dot(A.T)+mu 
 
    return(X.T)

In [44]:
a = genData(10000)


In [45]:
a

array([[-1.09093971, -4.72961093,  0.72427282, ..., -3.05744489,
        -5.06429066, -0.2168969 ],
       [ 3.73248998, -2.17624336,  7.86994539, ...,  1.72217391,
        -3.38719236,  7.61074706]])

In [38]:
q4 = genData(1000)

In [46]:
grm = q4.T.dot(q4)

In [47]:
grm.shape

(1000, 1000)

In [33]:
x = genData(10000)
muhat = np.mean(res,axis=1).reshape((2,1))
tmp = x-muhat
sig = tmp.dot(tmp.T)/(res.shape[1] - 1)
print(sig)

[[ 5.08919268 11.2396911 ]
 [11.2396911  25.60980733]]


In [31]:
np.cov(res)

array([[ 5.08919268, 11.2396911 ],
       [11.2396911 , 25.60980733]])

In [26]:
# Covariance
def cov(x, y):
    xbar, ybar = x.mean(), y.mean()
    return np.sum((x - xbar)*(y - ybar))/(len(x) - 1)

# Covariance matrix
def cov_mat(X):
    return np.array([[cov(X[0], X[0]), cov(X[0], X[1])], \
                     [cov(X[1], X[0]), cov(X[1], X[1])]])

# Calculate covariance matrix 
cov_mat(res) # (or with np.cov(X.T))

array([[ 5.08919268, 11.2396911 ],
       [11.2396911 , 25.60980733]])

In [80]:
import csv
with open("new_file.csv","r") as csvfile:
    csvr = csv.reader(csvfile, delimiter=',')
    for r in csvr:
        try:
            print(r)
            scr = "/home/jy/projects/yolo/dataprepare/notvalid/"+r[0]+"/"
            dst = "/home/jy/projects/yolo/dataprepare/notvalid/"+r[1]+"/"+r[0]
            if os.path.isdir(dst) == False:
                os.makedirs(dst)
            shutil.move(scr, dst)
        except FileNotFoundError:
            pass
 

['D16888040', '2']
['D54809356', '1']
['D54809659', '6']
['D54809682', '1']
['D54809832', '5']
['D70001524', '6']
['D70007834', '3']
['D70008083', '1']
['D70008380', '5']
['D70008439', '1']
['D70016784', '2']
['D81139009', '2']
['D81139192', '1']
['D81139772', '1']
['D81139831', '1']
['D81139985', '1']
['D81140045', '1']
['D81140084', '4']
['D81140257', '1']
['D81141169', '2']
['D81141211', '5']
['D81141212', '4']
['D81141229', '1']
['D81141235', '4']
['D81141237', '3']
['D81141264', '3']
['D81141269', '1']
['D81141274', '5']
['D81141283', '2']
['D81141342', '5']
['D81141370', '1']
['D81141409', '3']
['D81141420', '1']
['D81141435', '1']
['D81141444', '1']
['D81141454', '3']
['D81141494', '1']
['D81141499', '1']
['D81141530', '2']
['D81141532', '1']
['D81141543', '5']
['D81141548', '2']
['D81141557', '3']
['D81141561', '5']
['D81141570', '2']
['D81141576', '5']
['D81141577', '1']
['D81141585', '3']
['D81141588', '1']
['D81141590', '1']
['D81141591', '3']
['D81141607', '1']
['D81141623'

In [ ]:
for root,dir,files in os.walk("/home/jy/projects/yolo/dataprepare/valid/"):
    for file in files:
        if file.split(".")[-1] == "mp4":
            path = os.path.join(root,file)
            vcap = cv2.VideoCapture(path)  # 0=camera

            if vcap.isOpened():
                width = vcap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
                height = vcap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
 
                print('width, height:', width, height)

                fps = vcap.get(cv2.CAP_PROP_FPS)
                print('fps:', fps)  # float
                # print(cv2.CAP_PROP_FPS) # 5

                frame_count = vcap.get(cv2.CAP_PROP_FRAME_COUNT)
                print('frames count:', frame_count)  # float
                # print(cv2.CAP_PROP_FRAME_COUNT) # 7
             
            
            if width!=768.0 :
                print(''.join(path.split('.')[:-1])+"_resized.mp4")
                os.system("ffmpeg -i "+path+" -s 768x432 -c:a copy "+''.join(path.split('.')[:-1])+"_resized.mp4")

width, height: 768.0 432.0
fps: 9.994367046989199
frames count: 8481.0
width, height: 768.0 432.0
fps: 9.993819611556011
frames count: 8360.0
width, height: 768.0 432.0
fps: 9.99307870248399
frames count: 12600.0
width, height: 768.0 432.0
fps: 9.99191249695939
frames count: 4806.0
width, height: 768.0 432.0
fps: 9.98125039087819
frames count: 17406.0
width, height: 768.0 432.0
fps: 9.9945458335594
frames count: 7000.0
width, height: 768.0 432.0
fps: 9.986958407534221
frames count: 13493.0
width, height: 768.0 432.0
fps: 9.99485864348233
frames count: 6493.0
width, height: 768.0 432.0
fps: 9.993084474534824
frames count: 11021.0
width, height: 768.0 432.0
fps: 9.982562731440451
frames count: 19560.0
width, height: 768.0 432.0
fps: 9.994218124190947
frames count: 8539.0
width, height: 768.0 432.0
fps: 9.992961688041836
frames count: 4160.0
width, height: 768.0 432.0
fps: 9.993830083365618
frames count: 8358.0
width, height: 768.0 432.0
fps: 9.987096111799474
frames count: 4280.0
width, 

width, height: 768.0 432.0
fps: 9.994287029852401
frames count: 11721.0
width, height: 768.0 432.0
fps: 9.989076235539443
frames count: 20081.0
width, height: 768.0 432.0
fps: 9.993807664628854
frames count: 8360.0
width, height: 768.0 432.0
fps: 9.99398135738093
frames count: 9880.0
width, height: 768.0 432.0
fps: 9.99452473862145
frames count: 4600.0
width, height: 768.0 432.0
fps: 10.01312511078669
frames count: 10055.0
width, height: 768.0 432.0
fps: 9.99311914465675
frames count: 4720.0
width, height: 768.0 432.0
fps: 9.996089861287858
frames count: 5335.0
width, height: 768.0 432.0
fps: 9.99312972331522
frames count: 4640.0
width, height: 768.0 432.0
fps: 9.993978743478104
frames count: 10800.0
width, height: 768.0 432.0
fps: 9.994097840855263
frames count: 19620.0
width, height: 768.0 432.0
fps: 9.994258542091785
frames count: 8820.0
width, height: 768.0 432.0
fps: 9.993932255416354
frames count: 17080.0
width, height: 768.0 432.0
fps: 9.993541636001762
frames count: 6293.0
widt

In [40]:
for root,dir,files in os.walk("valid"):
    for d in dir:
        for roots,dir,files in os.walk(os.path.join(root,d)):
            if len(files)==1:
                print(files)
    break

['D88629692_2020_09_24_06_11_12.mp4']
['D88628149_2020_09_26_04_23_27.mp4']
['D88628855_2020_09_26_01_20_17.mp4']
['D81139009_2020_09_26_10_31_50.mp4']
['D81141741_2020_09_23_03_14_42.mp4']
['D81141169_2020_09_26_09_08_44_resized.mp4']
['D81141548_2020_09_28_08_53_30.mp4']
['D81142461_2020_09_26_06_23_37.mp4']
['D81141283_2020_09_26_07_05_29.mp4']
['E57379043_2020_09_25_00_10_02_resized.mp4']
['stitched-video_resized.mp4']
['D70016784_2020_09_26_01_19_33.mp4']
['D81141530_2020_09_26_02_42_12.mp4']
['D81142925_2020_09_26_10_30_22.mp4']
['D81141274_2020_09_28_07_52_25.mp4']
['E57378949_2020_09_27_04_10_08_resized.mp4']
['D88628584_2020_09_26_06_25_21.mp4']
['E57378942_2020_09_26_01_30_03_resized.mp4']
['D88629483_2020_09_26_01_45_22.mp4']
['D81141342_2020_09_27_00_12_41.mp4']
['D81145782_2020_09_26_09_20_19.mp4']
['D81142434_2020_09_27_00_30_47.mp4']
['D54809832_2020_09_28_00_16_25.mp4']
['D81142153_2020_09_26_01_21_20.mp4']
['D70008380_2020_09_28_09_14_01.mp4']
['stitched-video.mp4']
['

In [8]:
bracket = [4160,
12518,
16798,
30294,
36894]
names = ['valid/2/D88628855/D88628855/D88628855_2020_09_26_01_20_17.mp4', 'valid/2/D81139009/D81139009/D81139009_2020_09_26_10_31_50.mp4', 'valid/2/D81141741/D81141741/D81141741_2020_09_23_03_14_42.mp4', 'valid/2/D81141169/D81141169/D81141169_2020_09_26_09_08_44_resized.mp4', 'valid/2/D81141548/D81141548/D81141548_2020_09_28_08_53_30.mp4']

for root,dir,files in os.walk("unzips/train/task_vaild_2_0-2020_10_27_07_56_32-yolo/obj_train_data"):
    for file in files:
        frame_num = int(file.split(".")[0].split("_")[1])
        for i,x in enumerate(bracket):
            if frame_num<bracket[0]:
                newname = names[0].split('/')[2]+"_"+names[0].split('/')[1]+"_"+file
            elif frame_num>x and frame_num<bracket[i+1]:
                newname = names[i+1].split('/')[2]+"_"+names[i+1].split('/')[1]+"_"+file
        os.rename(os.path.join(root,file),os.path.join(root,newname))
        # print(newname)

In [7]:
for root,dir,files in os.walk("valid"):
    for d in dir:
#         print(os.path.join(root,d))
        file_list =[]
        dstpath = "vaild"

        for roots,dir,files in os.walk(os.path.join(root,d)):
#             for f in files:
            if len(files)==1:
                
                file_list.append(os.path.join(roots,files[0]))
            elif len(files)>1:
                fname = "stitched-video.mp4"
                src = os.path.join(roots,fname)
                dst = dstpath + d+"_"+roots.split("/")[-1]+"_"+fname
 
                file_list.append(dst)
#                 copyfile(src, dst)
        if len(file_list)>0:
             
            composite_list = [file_list[x:x+5] for x in range(0, len(file_list),5)]
            for ind, li in enumerate(composite_list):
                open("list.txt", 'w').close()
                tmp = 0
                for i in li:
#                     f = open("list.txt", "a")
#                     f.write("file "+"'"+i+"'" )
#                     f.write("\n")
#                     f.close()
                    cap = cv2.VideoCapture(i) 
                    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    tmp+=length
                    print(tmp)
                for unzroot,unzdir,unzfiles in os.walk("/home/jy/projects/imgs/dataprepare/unzips/"):
                    for ds in unzdir:
#                         print("task_vaild_"+d+"_"+str(ind))
                        if "task_vaild_"+d+"_"+str(ind) in ds:
                            print("task_vaild_"+d+"_"+str(ind))
                            print(ds)
#                     print(length)
                print(li) 
                 
                print("ffmpeg -f concat -safe 0 -i list.txt -c copy "+d+"_"+str(ind)+".mp4\n")
            
#                 os.system("ffmpeg -f concat -safe 0 -i list.txt -c copy "+d+"_"+str(ind)+".mp4")
    break
#     for file in files:
#         label = root.split("/")[-3]
#         print(label)
#         print(file)

8481
16841
16841
16841
16841
task_vaild_4_0
task_vaild_4_0-2020_10_28_02_30_10-yolo
['valid/4/D88629692/D88629692/D88629692_2020_09_24_06_11_12.mp4', 'valid/4/D88628149/D88628149/D88628149_2020_09_26_04_23_27.mp4', 'vaild4_D81141235_stitched-video.mp4', 'vaild4_D88628648_stitched-video.mp4', 'vaild4_D88629079_stitched-video.mp4']
ffmpeg -f concat -safe 0 -i list.txt -c copy 4_0.mp4

4160
12518
16798
30294
36894
task_vaild_2_0
task_vaild_2_0-2020_10_27_07_56_32-yolo
['valid/2/D88628855/D88628855/D88628855_2020_09_26_01_20_17.mp4', 'valid/2/D81139009/D81139009/D81139009_2020_09_26_10_31_50.mp4', 'valid/2/D81141741/D81141741/D81141741_2020_09_23_03_14_42.mp4', 'valid/2/D81141169/D81141169/D81141169_2020_09_26_09_08_44_resized.mp4', 'valid/2/D81141548/D81141548/D81141548_2020_09_28_08_53_30.mp4']
ffmpeg -f concat -safe 0 -i list.txt -c copy 2_0.mp4

8000
19680
34587
62432
69939
task_vaild_2_1
task_vaild_2_1-2020_10_27_08_30_20-yolo
['valid/2/D81142461/D81142461/D81142461_2020_09_26_06_23_3

In [102]:
for root,dir,files in os.walk("/home/jy/projects/yolo/dataprepare/notvaild/"):
    for d in dir:
        print(os.path.join(root,d))
        file_list =[]
        dstpath = "/home/jy/projects/yolo/dataprepare/needtobelabeled/notvaild/"
        open("list.txt", 'w').close()

        for roots,dir,files in os.walk(os.path.join(root,d)):
#             for f in files:
            
            if len(files)==1:
                
                file_list.append(os.path.join(roots,files[0]))
            elif len(files)>1:
                fname = files[1]
                src = os.path.join(roots,fname)
                dst = dstpath + d+"_"+fname
                file_list.append(os.path.join(roots,fname))
#                 file_list.append(os.path.join(roots,fname))
#                 copyfile(src, dst)
         
        if len(file_list)>0:
            composite_list = [file_list[x:x+5] for x in range(0, len(file_list),5)]
            for ind, li in enumerate(composite_list):
                 
                for i in li:
                    f = open("list.txt", "a")
                    f.write("file "+"'"+i+"'" )
                    f.write("\n")
                    f.close()
#             print(composite_list) 
                print("ffmpeg -f concat -safe 0 -i list.txt -c copy "+d+"_"+str(ind)+".mp4")
                os.system("ffmpeg -f concat -safe 0 -i list.txt -c copy "+d+"_"+str(ind)+".mp4")
    break
#     for file in files:
#         label = root.split("/")[-3]
#         print(label)
#         print(file)

/home/jy/projects/yolo/dataprepare/notvaild/4
ffmpeg -f concat -safe 0 -i list.txt -c copy 4_0.mp4
/home/jy/projects/yolo/dataprepare/notvaild/2
/home/jy/projects/yolo/dataprepare/notvaild/5
ffmpeg -f concat -safe 0 -i list.txt -c copy 5_0.mp4
ffmpeg -f concat -safe 0 -i list.txt -c copy 5_1.mp4
/home/jy/projects/yolo/dataprepare/notvaild/3
ffmpeg -f concat -safe 0 -i list.txt -c copy 3_0.mp4
/home/jy/projects/yolo/dataprepare/notvaild/1
ffmpeg -f concat -safe 0 -i list.txt -c copy 1_0.mp4
/home/jy/projects/yolo/dataprepare/notvaild/7
ffmpeg -f concat -safe 0 -i list.txt -c copy 7_0.mp4
/home/jy/projects/yolo/dataprepare/notvaild/6


In [21]:
for root,dir,files in os.walk("secondbatch/work_done_zip"):
    for f in files:
        dirname = f.split(".")[0]
#         os.rename("secondbatch/work_done_zip/"+f,"secondbatch/work_done_zip/"+f.split(" ")[0]+".zip")
 
    
        subprocess.run(shlex.split("mkdir "+"secondbatch/secondbatch_unzips/"+dirname))
        subprocess.run(shlex.split("unzip secondbatch/work_done_zip/"+f+" -d secondbatch/secondbatch_unzips/"+dirname))
        

In [8]:
for root,dir,files in os.walk("work_done_zip"):
    for f in files:

#         os.rename(os.path.join(root,f),os.path.join(root,f.split(" ")[0])+".zip")
        dirname = f.split(".")[0]
    
        subprocess.run(shlex.split("mkdir "+"secondbatch_unzips/"+dirname))
#         subprocess.run(shlex.split("unzip "+  os.path.join(root,f) + " -d " +"secondbatch_unzips/"+"/".join(os.path.join(root,dirname).split("/")[1:])))

#         os.system("mkdir "+"secondbatch_unzips/"+"/".join(os.path.join(root,dirname).split("/")[1:]))
#         print("unzip "+  os.path.join(root,f) + " -d " +"secondbatch_unzips/"+"/".join(os.path.join(root,dirname).split("/")[1:]))
#         os.system("unzip "+  os.path.join(root,f) + " -d " +"secondbatch_unzips/"+"/".join(os.path.join(root,dirname).split("/")[1:]))


In [5]:
for root,dir,files in os.walk("unzips"):
    if len(dir)==1 and dir[0]=="obj_train_data":
        dirpath = os.path.join(root,"obj_train_data")
        dstpath = os.path.join(root,"obj_train_data_dst")
        os.system("mkdir "+dstpath)
        
        for r,d,fs in os.walk(dirpath):
            for f in fs:
                if f.split(".")[-1]=="txt":
                    with open(f,"r") as txt:
                        
                    
        
        
#     for d in dir
#     for f in files:
#         if f.split(".")[-1]=="txt":
            

unzips/JYLABEL/task_vaild_5_342-2020_10_28_06_12_24-yolo/obj_train_data
unzips/JYLABEL/task_train2-2020_10_28_01_41_16-yolo/obj_train_data
unzips/JYLABEL/task_train8-2020_10_28_02_24_22-yolo/obj_train_data
unzips/JYLABEL/task_vaild_5_832-2020_10_28_06_23_25-yolo/obj_train_data
unzips/JYLABEL/task_train7-2020_10_28_02_19_47-yolo/obj_train_data
unzips/JYLABEL/task_vaild_5_434-2020_10_28_06_18_43-yolo/obj_train_data
unzips/JYLABEL/task_test2-2020_10_27_10_41_23-yolo/obj_train_data
unzips/JYLABEL/task_vaild_2_1-2020_10_27_08_30_20-yolo/obj_train_data
unzips/JYLABEL/task_vaild_4_d81141235_stitched-2020_10_28_08_08_43-yolo/obj_train_data
unzips/JYLABEL/task_notvaild_5_sink1-2020_10_27_07_13_44-yolo/obj_train_data
unzips/JYLABEL/task_vaild_2_2-2020_10_27_08_40_00-yolo/obj_train_data
unzips/JYLABEL/task_vaild_5_d81142872_stitched-2020_10_28_06_56_03-yolo/obj_train_data
unzips/JYLABEL/task_notvaild_7_0-2020_10_27_07_01_05-yolo/obj_train_data
unzips/JYLABEL/task_train3-2020_10_28_01_47_33-yolo/o